# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apagar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [2]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [3]:
# Contar o número de registros e registros não duplicados
num_registros = sinasc.shape[0]
num_registros_unicos = sinasc.drop_duplicates().shape[0]

# Verificar se há duplicados
tem_duplicados = num_registros != num_registros_unicos

# Contar o número de valores faltantes por variável
missing_values = sinasc.isnull().sum()

# Selecionar colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionada = sinasc[colunas_interesse]

# Refazer a contagem de valores faltantes na nova base
missing_values_selecionada = sinasc_selecionada.isnull().sum()

# Exibir resultados
print(f"Total de registros: {num_registros}")
print(f"Total de registros únicos: {num_registros_unicos}")
print(f"Há duplicados: {tem_duplicados}")
print("\nValores faltantes por variável na base original:\n", missing_values)
print("\nValores faltantes por variável na base selecionada:\n", missing_values_selecionada)


Total de registros: 27028
Total de registros únicos: 27028
Há duplicados: False

Valores faltantes por variável na base original:
 ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

Valores faltantes por variável na base selecionada:
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [4]:
# Selecionar colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionada = sinasc[colunas_interesse]

# Remover registros com Apgar5 não preenchido
sinasc_selecionada = sinasc_selecionada.dropna(subset=['APGAR5'])

# Substituir valores faltantes em ESTCIVMAE e CONSULTAS por 9
sinasc_selecionada['ESTCIVMAE'] = sinasc_selecionada['ESTCIVMAE'].fillna(9)
sinasc_selecionada['CONSULTAS'] = sinasc_selecionada['CONSULTAS'].fillna(9)

# Substituir valores faltantes em QTDFILVIVO por zero
sinasc_selecionada['QTDFILVIVO'] = sinasc_selecionada['QTDFILVIVO'].fillna(0)

# Analisar valores faltantes restantes
missing_values_restantes = sinasc_selecionada.isnull().sum()
print("\nValores faltantes por variável após preenchimento inicial:\n", missing_values_restantes)

# Preencher valores faltantes restantes
# LOCNASC: Preencher com o valor mais frequente
locnasc_mode = sinasc_selecionada['LOCNASC'].mode()[0]
sinasc_selecionada['LOCNASC'] = sinasc_selecionada['LOCNASC'].fillna(locnasc_mode)

# IDADEMAE: Preencher com a média
idade_mae_mean = sinasc_selecionada['IDADEMAE'].mean()
sinasc_selecionada['IDADEMAE'] = sinasc_selecionada['IDADEMAE'].fillna(idade_mae_mean)

# ESCMAE: Preencher com o valor mais frequente
escmae_mode = sinasc_selecionada['ESCMAE'].mode()[0]
sinasc_selecionada['ESCMAE'] = sinasc_selecionada['ESCMAE'].fillna(escmae_mode)

# GESTACAO: Preencher com o valor mais frequente
gestacao_mode = sinasc_selecionada['GESTACAO'].mode()[0]
sinasc_selecionada['GESTACAO'] = sinasc_selecionada['GESTACAO'].fillna(gestacao_mode)

# GRAVIDEZ: Preencher com o valor mais frequente
gravidez_mode = sinasc_selecionada['GRAVIDEZ'].mode()[0]
sinasc_selecionada['GRAVIDEZ'] = sinasc_selecionada['GRAVIDEZ'].fillna(gravidez_mode)

# Criar categorização para APGAR5
def categorize_apgar(apgar5):
    if apgar5 >= 8:
        return 'normal'
    elif apgar5 >= 6:
        return 'asfixia_leve'
    elif apgar5 >= 4:
        return 'asfixia_moderada'
    else:
        return 'asfixia_severa'

sinasc_selecionada['APGAR5_CATEGORIA'] = sinasc_selecionada['APGAR5'].apply(categorize_apgar)

# Calcular frequências da categorização
frequencias_apgar5_categoria = sinasc_selecionada['APGAR5_CATEGORIA'].value_counts()
print("\nFrequências da categorização APGAR5:\n", frequencias_apgar5_categoria)

# Renomear variáveis para snake case
sinasc_selecionada.rename(columns={
    'LOCNASC': 'locnasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'estcivmae',
    'ESCMAE': 'escmae',
    'QTDFILVIVO': 'qtdfilvivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5',
    'APGAR5_CATEGORIA': 'apgar5_categoria'
}, inplace=True)

# Exibir DataFrame final
print("\nDataFrame final:\n", sinasc_selecionada.head())



Valores faltantes por variável após preenchimento inicial:
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

Frequências da categorização APGAR5:
 APGAR5_CATEGORIA
normal              26463
asfixia_leve          320
asfixia_severa         74
asfixia_moderada       68
Name: count, dtype: int64

DataFrame final:
    locnasc  idade_mae  estcivmae           escmae  qtdfilvivo  \
0        1         19        5.0      8 a 11 anos         0.0   
1        1         29        2.0      8 a 11 anos         1.0   
2        1         37        9.0      8 a 11 anos         2.0   
3        1         30        5.0  12 anos ou mais         0.0   
4        1         30        2.0      8 a 11 anos         1.0   

          gestacao gravidez  consultas  apgar5 apgar5_categoria  
0  37 a 41 semanas    Única          4    10.0           normal  
1  37 a 41 semanas    Única

In [5]:
# 6) Seu código aqui


In [6]:
# 7) seu código aqui


In [7]:
# 8) seu código aqui


In [8]:
# 9) seu código aqui
